In [1]:
!pip install scikit-surprise

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

pd.options.display.max_columns=None
#pd.set_option('display.max_colwidth', None)

import warnings; warnings.simplefilter('ignore')
from tabulate import tabulate

In [3]:
md = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')

In [4]:
md=md[md['imdb_id'].notnull()]

In [5]:
md['genres']=md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])

In [6]:
credits=pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv')
keywords=pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
md=md.drop([19730, 29503, 35587])
md['id']=md['id'].astype('int')
keywords['id']=keywords['id'].astype('int')
credits['id']=credits['id'].astype('int')
md=md.merge(credits,on='id')
md=md.merge(keywords,on='id')

In [7]:
md['cast']=md['cast'].apply(literal_eval)
md['keywords']=md['keywords'].apply(literal_eval)
md['crew']=md['crew'].apply(literal_eval)

In [8]:
md['cast']=md['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else[])
md['cast']=md['cast'].apply(lambda x: x[:4] if len(x)>=4 else x)
md['keywords']=md['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else[])

In [9]:
md=md[md['imdb_id'].notnull()]

In [10]:
def get_director(x):
    for i in x:
        if i['job']=='Director':
            return i['name']
        return np.nan

In [11]:
md['director']=md['crew'].apply(get_director)

In [12]:
md['cast']=md['cast'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])
md['director']=md['director'].astype('str').apply(lambda x: str.lower(x.replace(" ","")))

In [13]:
s=md.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1,drop=True)
s=s.value_counts()

In [14]:
s=s[s>1]

In [15]:
stemmer=SnowballStemmer('english')

In [16]:
def filter_keywords(x):
    words=[]
    for i in x:
        if i in s:
            words.append(i)
    return words

In [17]:
md['original_language']=md['original_language'].astype('str')

In [18]:
md['ol']=md['original_language']

In [19]:
md['keywords']=md['keywords'].apply(filter_keywords)
md['keywords']=md['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
md['keywords']=md['keywords'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])
md['director']=md['director'].apply(lambda x: [x,x,x])
md['original_language']=md['original_language'].apply(lambda x: [x,x])

In [20]:
md['soup']=md['keywords']+md['cast']+md['director']+md['genres']+md['original_language']

In [21]:
md['soup']=md['soup'].apply(lambda x: ' '.join(x))

In [22]:
dataset_small=pd.read_csv('/kaggle/input/the-movies-dataset/links_small.csv')
dataset_small=dataset_small[dataset_small['tmdbId'].notnull()]['tmdbId'].astype('int')
smd=md[md['id'].isin(dataset_small)]

In [23]:
count=CountVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
count_matrix=count.fit_transform(smd['soup'])

In [24]:
smd=smd.reset_index()

In [25]:
def Content_Based_Recomendations(imdb):
    try:
        IDX=smd.index[smd['imdb_id']==imdb][0]
    except:
        return "NA"
    cos_sim=cosine_similarity(count_matrix[IDX],count_matrix).flatten()
    sim_scores=list(enumerate(cos_sim))
    sim_scores=sorted(sim_scores, key=lambda x:x[1], reverse=True)
    sim_scores=sim_scores[1:11]
    response = {'result': [smd.iloc[t_vect[0]]['original_title'] for t_vect in sim_scores]}
    response = pd.DataFrame(response)
    
    return response

In [34]:
Content_Based_Recomendations('tt1187043') #https://www.imdb.com/title/tt1187043/
#3 Idiots

In [35]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

**Simple Recommender**

In [38]:
def SR_genre_lang(genres, language):
    genres=genres[0]
    language=language[0]
    df=gen_md[gen_md['genre'].isin(genres)]
    df=df[df['ol'].isin(language)]
    SR=md[md['imdb_id'].isin(df['imdb_id'])]
    vote_counts_SR = SR[SR['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages_SR = SR[SR['vote_average'].notnull()]['vote_average'].astype('int')
    C_SR = vote_averages_SR.mean()
    m_SR = vote_counts_SR.quantile(0.90)
    qualified_SR = SR[(SR['vote_count'] >= m_SR) & (SR['vote_count'].notnull()) & (SR['vote_average'].notnull())]
    qualified_SR['vote_count'] = qualified_SR['vote_count'].astype('int')
    qualified_SR['vote_average'] = qualified_SR['vote_average'].astype('int')
    qualified_SR['wr'] = ((qualified_SR['vote_count']*qualified_SR['vote_average'])+(m_SR*C_SR))/(qualified_SR['vote_count']+m_SR)
    qualified_SR=qualified_SR.sort_values('wr',ascending=False)
    qualified_SR=qualified_SR.head(10)
    response={'result': qualified_SR['original_title']}
    return pd.DataFrame(response)
    ##return qualified_SR[['imdb_id']]

In [40]:
SR_genre_lang(pd.DataFrame(['Action','Romance']),pd.DataFrame(['hi']))